In [1]:
#pip install git+https://github.com/deanmalmgren/textract.git

In [2]:
import textract

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
#extraigo texto
archivo= 'tresalia\Edo Cta\cuenta_perfiles_banamex.pdf'

In [5]:
text = textract.process('tresalia\Edo Cta\cuenta_perfiles_banamex.pdf')

In [6]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [7]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [8]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,cffibanamex@
1,El Banco Nacional de México
2,Página 1 de 11
3,g NOMBRE APELLIDO P APELLIDO M
4,"..,. DIRECCIÓN ""' COLONIA"


In [9]:
#extraigo nombre de cliente
cliente_nombre_list=[]
for index,row in information_pdf.iterrows():
    if row["informacion"].startswith("g"):
        cliente_nombre=row[0]
        cliente_nombre_list.append(cliente_nombre)
    

In [10]:
cliente_nombre_list

['g NOMBRE APELLIDO P  APELLIDO M']

In [11]:
nombre=(cliente_nombre_list[0].replace('g','')).strip()

In [12]:
nombre

'NOMBRE APELLIDO P  APELLIDO M'

In [13]:
#obtengo periodo

In [14]:
periodo_list=[]
for index,row in information_pdf.iterrows():
    if row["informacion"].startswith("Período") or row["informacion"].startswith("período") or row["informacion"].startswith("Periodo") or row["informacion"].startswith("periodo"):
        periodo=row[0]
        periodo_list.append(periodo)

In [15]:
periodo

'Período del 5 de septiembre al 4 de octubre del 2020'

In [16]:
periodo=[]
for i in pdf:
    if i.startswith("Período") or i.startswith("período") or i.startswith("Periodo") or i.startswith("periodo"):
        periodo.append(i)
        
print(periodo)

['Período del 5 de septiembre al 4 de octubre del 2020']


In [17]:
informacion=[]
for i in range(len(pdf)):
    if re.search('contrato',pdf[i]):
        informacion.append(pdf[i])
        informacion.append(pdf[i+1])

print(informacion)

['Número de contrato Número de sucursal Número de cuenta de cheques Número de Tarjeta de Débito CLABE lnterbancaria Número de cliente RFC', '9999999999 9999 99999999999 9999999999999999 999999999999999999 99999999 XXXX999999X99', 'Número de contrato', '9999999999', 'Tambien puedes realizar dep6sitos en cajeros automaticos y sucursales Cltibanamex. En caso de no estar de acuerdo con las modificaciones senaladas, podras solicitar la terminaci6n del Contrato dentro de los 30 dfas posteriore a Que surta electo la presente notificaci6n, por lo que durante ese pedodo nose aplicara el cobra de las comisiones mencionadas previamente. Se entendera que aceptas las modificaciones efectuadas a las comisiones, si no solicitas la terminaci6n del contrato dentro del plazo seiialado. Tu derecho a dar por terminado el Contrato en cualquier otro momenta seguira siendo vigente.', 'Atentamente.', 'Tu contrato de Banca lectr6nica tiene na nueva actualizaci6n', 'Te inlormarnos que tu contrato de prestacion 

In [18]:
nombre=informacion[0].split('Número')
nombre.pop(0)
nombre[3]='Tarjeta de Débito'
nombre[4]='CLABE'
nombre.append('# cliente')
nombre.append('RFC')


In [19]:
nombre

[' de contrato ',
 ' de sucursal ',
 ' de cuenta de cheques ',
 'Tarjeta de Débito',
 'CLABE',
 '# cliente',
 'RFC']

In [20]:
valores=informacion[1].split(' ')

In [21]:
val_dict= dict(zip(nombre,valores))

In [22]:
val_dict

{' de contrato ': '9999999999',
 ' de sucursal ': '9999',
 ' de cuenta de cheques ': '99999999999',
 'Tarjeta de Débito': '9999999999999999',
 'CLABE': '999999999999999999',
 '# cliente': '99999999',
 'RFC': 'XXXX999999X99'}

In [23]:
pdf

['cffibanamex@',
 'El Banco Nacional de México',
 'Página 1 de 11',
 'g NOMBRE APELLIDO P  APELLIDO M',
 '..,. DIRECCIÓN "\' COLONIA',
 'IN',
 '00000 DELEGACIÓN, CIUDAD DE MEXICO C.R.11801',
 'Estado de Cuenta',
 'Cuenta Perfiles',
 'Período del 5 de septiembre al 4 de octubre del 2020',
 'Fecha de Corte',
 '02 de octubre de 2020',
 'Número de contrato Número de sucursal Número de cuenta de cheques Número de Tarjeta de Débito CLABE lnterbancaria Número de cliente RFC',
 '9999999999 9999 99999999999 9999999999999999 999999999999999999 99999999 XXXX999999X99',
 'GAT Nominal y Real antes de impuestos Interés aplicable o Rendimientos Comisiones efectivamente cobradas',
 'Comportamiento',
 'No Aplica $0.00',
 '$105.00',
 'Resumen En Pesos Moneda Nacional',
 'Saldo anterior Depósitos Retiros en efectivo Otros cargos Saldo al corte Saldo promedio mensuaf',
 '1,291.41 0.00 0.00',
 '5,538.45 5,647.96 2,065.77',
 'Saldo anterior',
 'o',
 'Depósitos',
 'Retiros en efectivo',
 '$0',
 'Comisiones $

In [24]:
#movimientos
movimientos=[]
for i in range(len(pdf)):
    if pdf[i].startswith('Detalle'):
        movimientos.append(i)

In [25]:
movimientos

[94, 124, 160]

In [26]:
#fin de movimientos
fin=[]
for i in range(len(pdf)):
    if pdf[i].startswith('Resumen Operaciones Tarjeta'):
        fin.append(i)

In [27]:
fin

[190]

In [28]:
tabla_pag_1=pdf[movimientos[0]:movimientos[1]]

# PAGINA 1

In [29]:
#fechas
fecha_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('fecha',tabla_pag_1[i].lower()):
        fecha_lista_pag_1.append(tabla_pag_1[i])
    elif re.match(r'\d{2} \w{3}',tabla_pag_1[i]):
        fecha_lista_pag_1.append(tabla_pag_1[i])

        
fecha_lista_pag_1

['FECHA 05 SEP 07 SEP 08 SEP', '10 SEP 10 SEP 10 SEP 18 SEP 18 SEP', '18 SEP']

In [30]:
#separo
fechas_lista_2_pag_1=[]

for i in fecha_lista_pag_1:
    fechas_lista_2_pag_1.append(re.findall(r'\d{2} \w{3}',i))

In [31]:
#aplano lista de listas
fechas_pag_1 = []
for i in fechas_lista_2_pag_1:
    for j in i:
        fechas_pag_1.append(j)

In [32]:
fechas1_pag_2=fechas_pag_1.pop(-1)

In [33]:
len(fechas_pag_1)

8

In [34]:
#conceptos
tabla_string1=" ".join(tabla_pag_1)

In [35]:
conceptos_p1=re.split('SALDO |EXCENSINCION |COB|XXXX XXXX|GN',tabla_string1)
conceptos_p1

['Detalle de Operaciones En pesos Moneda Nacional FECHA 05 SEP 07 SEP 08 SEP 10 SEP 10 SEP 10 SEP 18 SEP 18 SEP 18 SEP CONCEPTO ',
 'ANTERIOR ',
 'RO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION ',
 'RO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 ',
 'RO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:30 ',
 'RO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 AUT 00000000 HORA 14:49 ',
 'RO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:50 ',
 'C PLAZA LILAS MAX0611157H8 9999 999999999999999999999999 99999999 CAJA 0093 AUT 99999999 99999999 HORA 06:48 SUC 9999 ',
 ' & XXX XX 000000000 0000 suc 0342 CAJA 0000 AUT 00000000 HORA 06:48 00000000 00000000000000000000000 00000000 suc 0000 CAJA 0000 AUT 00000000 HORA 06:48 LILAS 000000000 0006475 Envía y recibe dinero al instante sin dar de alta una cuenta, hazlo desde CoDi(!) en Citibanamex Móvil Maneja tu dinero desde la 

In [36]:
conceptos_pag_1=[]
for i in range(len(conceptos_p1)):
    if conceptos_p1[i].startswith('ANTERIOR'):
        j='SALDO '+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('RO COMISION'):
        j='EXCENSION COB'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('RO'):
        j='COB'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('C'):
        j='GN'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith(' &'):
        j='XXXX XXXX'+conceptos_p1[i]
        conceptos_pag_1.append(j)
conceptos_pag_1

['SALDO ANTERIOR ',
 'COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION ',
 'EXCENSION COBRO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 ',
 'COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:30 ',
 'COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 AUT 00000000 HORA 14:49 ',
 'COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:50 ',
 'GNC PLAZA LILAS MAX0611157H8 9999 999999999999999999999999 99999999 CAJA 0093 AUT 99999999 99999999 HORA 06:48 SUC 9999 ',
 'XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJA 0000 AUT 00000000 HORA 06:48 00000000 00000000000000000000000 00000000 suc 0000 CAJA 0000 AUT 00000000 HORA 06:48 LILAS 000000000 0006475 Envía y recibe dinero al instante sin dar de alta una cuenta, hazlo desde CoDi(!) en Citibanamex Móvil Maneja tu dinero desde la App. RETIROS DEPÓSITOS 2,000.00 950.00 1 686.00 1,326.00 2,029.70 350.00 000190.B25INDA0I

In [37]:
#retiros
retiros_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('retiros',tabla_pag_1[i].lower()):
        retiros_lista_pag_1.append(tabla_pag_1[i])
        retiros_lista_pag_1.append(tabla_pag_1[i+1])
        retiros_lista_pag_1.append(tabla_pag_1[i+2])
        retiros_lista_pag_1.append(tabla_pag_1[i+3])
        retiros_lista_pag_1.append(tabla_pag_1[i+4])
retiros_lista_pag_1

['RETIROS',
 'DEPÓSITOS',
 '2,000.00',
 '950.00 1 686.00 1,326.00 2,029.70',
 '350.00']

In [38]:
retiros1_1=[]
for i in retiros_lista_pag_1:
    if re.search(r'\d+', i):
        retiros1_1.append(i)

In [39]:
retiros1_1

['2,000.00', '950.00 1 686.00 1,326.00 2,029.70', '350.00']

In [40]:
retiros1_2=[]
for i in retiros1_1:
    retiros1_2.append(i.split('.00'))
    
retiros1_2

[['2,000', ''], ['950', ' 1 686', ' 1,326', ' 2,029.70'], ['350', '']]

In [41]:
retiros1_3=[]
for i in retiros1_2:
    for j in i:
        if re.search(r'\d+', j):
            retiros1_3.append(j+'.00')
        
retiros1_3

['2,000.00', '950.00', ' 1 686.00', ' 1,326.00', ' 2,029.70.00', '350.00']

In [42]:
#organizo retiros pagina 1
retiros_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('COBRO'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
    
    elif conceptos_pag_1[i].startswith('GNC'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
        
    elif conceptos_pag_1[i].startswith('XXX'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
    
    else:
        retiros_pag_1.append(0)
retiros_pag_1

[0,
 '2,000.00',
 0,
 '950.00',
 ' 1 686.00',
 ' 1,326.00',
 ' 2,029.70.00',
 '350.00']

In [43]:
#depositos
depositos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('depósitos',tabla_pag_1[i].lower()):
        depositos_lista_pag_1.append(tabla_pag_1[i])
        
depositos_lista_pag_1
#no hay depositos

['DEPÓSITOS']

In [44]:
depositos1=[]
for i in depositos_lista_pag_1:
    if re.search(r'\d+', i):
        depositos1.append(i)

depositos1_1=[]     
for i in depositos1:
    j=i.split(' ')
    for k in j:
        depositos1_1.append(k)

In [45]:
#organizo depositos
depositos_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('DEPOSITO') or conceptos_pag_1[i].startswith('INTERES'):
        depositos_pag_1.append(depositos1_1[contador])
        contador=contador+1
    else:
        depositos_pag_1.append('0')
depositos_pag_1

['0', '0', '0', '0', '0', '0', '0', '0']

In [46]:
#saldos
saldos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('saldo',tabla_pag_1[i].lower()):
        saldos_lista_pag_1.append(tabla_pag_1[i])
        saldos_lista_pag_1.append(tabla_pag_1[i+1])
        saldos_lista_pag_1.append(tabla_pag_1[i+2])
        
saldos_lista_pag_1

['SALDO ANTERIOR COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION COBRO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 COBRO DE CHEQUE NUMERO 000',
 'suc 0000',
 'CAJA 0000 AUT 00000000 HORA 14:30 COBRO DE CHEQUE NUMERO 174',
 'SALDO 1 291.41 9,291.41 9,291.41 8,341.41 6 655.41 5,329.41 3,299.71 2 949.71',
 '\x0ccffibanamex@',
 'NOMBRE APELLIDO P APELLIDO M']

In [47]:
saldos1_1=[]
for i in range(len(saldos_lista_pag_1)):
    if re.search(r'SALDO \d+', saldos_lista_pag_1[i]):
        saldos1_1.append(saldos_lista_pag_1[i])
        saldos1_1.append(saldos_lista_pag_1[i+1])
        
saldos1_1

['SALDO 1 291.41 9,291.41 9,291.41 8,341.41 6 655.41 5,329.41 3,299.71 2 949.71',
 '\x0ccffibanamex@']

In [48]:
saldos1_2=[]
for i in saldos1_1:
    j=re.split('SALDO|\.\d{2}',i)
    saldos1_2.append(j)
    
saldos1_3=[]
for i in saldos1_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos1_3.append(j)
            
saldos1_3

[' 1 291',
 ' 9,291',
 ' 9,291',
 ' 8,341',
 ' 6 655',
 ' 5,329',
 ' 3,299',
 ' 2 949']

In [49]:
saldos_pag_1= saldos1_3

# Pagina 2

In [50]:
tabla_pag_2=pdf[movimientos[1]:movimientos[2]]

In [101]:
#fechas
fecha_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('fecha',tabla_pag_2[i].lower()):
        fecha_lista_pag_2.append(tabla_pag_2[i])
    elif re.match(r'\d{2} \w{3}',tabla_pag_2[i]):
        fecha_lista_pag_2.append(tabla_pag_2[i])

        
fecha_lista_pag_2

['FECHA 21 SEP 21 SEP 21 SEP 22 SEP',
 '24 SEP 24 SEP 24 SEP 25 SEP 25 SEP',
 '25 SEP',
 '28 SEP']

In [102]:
#separo
fechas_lista_2_pag_2=[]

for i in fecha_lista_pag_2:
    fechas_lista_2_pag_2.append(re.findall(r'\d{2} \w{3}',i))

In [103]:
#aplano lista de listas
fechas_pag_2 = []
fechas_pag_2.append(fechas1_pag_2)
for i in fechas_lista_2_pag_2:
    for j in i:
        fechas_pag_2.append(j)

In [104]:
fechas1_pag_3=fechas_pag_2.pop(-1)

In [105]:
len(fechas_pag_2)

11

In [133]:
#conceptos
tabla_string_2=" ".join(tabla_pag_2)

In [134]:
conceptos_p2=re.split('5411|REPOS|COB|MARK|FLO|UBER.',tabla_string_2)
conceptos_p2

['Detalle de Operaciones En pesos Moneda Nacional FECHA 21 SEP 21 SEP 21 SEP 22 SEP 24 SEP 24 SEP 24 SEP 25 SEP 25 SEP 25 SEP 28 SEP CONCEPTO ',
 ' 75412910263975987909057 20200921 CAJA 0093 AUT 02985979 20200921 HORA 06:48 SUC 0342 ',
 'TERIA XXXX999999XX9 5441 20200923 75326900265000784368680 20200923 suc 0342 CAJA 0093 AUT 01011285 HORA 05:30 CITY ',
 'ET ',
 ' 75435290266536760465142 20200923 CAJA 0093 AUT 01011287 20200923 HORA 05:30 SUC 0342 ',
 'RES 5992 20200923 75412780265000108700221 20200923 suc 0342 CAJA 0093 AUT 01011289 HORA 05:30 TERMINAL 131210UU7 5811 suc 0342 CAJA 0093 AUT 00881422 HORA 05:30 20200924 75445500266001282364505 20200924 suc 0342 CAJA 0093 AUT 00881422 HORA 05:30 CITY ',
 'ET ',
 ' 75435290269536760467046 20200928 CAJA 0093 AUT 00906083 20200928 HORA 05:40 SUC 0342 ',
 'RO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451934 HORA 12:19 ',
 'RO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451948 HORA 12:32 ',
 'RO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0001 A

In [135]:
conceptos_pag_2=[]
for i in range(len(conceptos_p2)):
    if conceptos_p2[i].startswith(' 7541'):
        j='5411'+conceptos_p2[i]
        conceptos_pag_2.append(j)
    elif conceptos_p2[i].startswith('TERIA'):
        j='REPOS'+conceptos_p2[i]
        conceptos_pag_2.append(j)
    elif conceptos_p2[i].startswith('RES '):
        j='FLO'+conceptos_p2[i]
        conceptos_pag_2.append(j)
    elif conceptos_p2[i].startswith('RO'):
        j='COB'+conceptos_p2[i]
        conceptos_pag_2.append(j)
    elif conceptos_p2[i].startswith('ET'):
        j='MARKET'+conceptos_p2[i+1]
        conceptos_pag_2.append(j)
    elif conceptos_p2[i].startswith('COM'):
        j='UBER.'+conceptos_p2[i]
        conceptos_pag_2.append(j)
conceptos_pag_2

['5411 75412910263975987909057 20200921 CAJA 0093 AUT 02985979 20200921 HORA 06:48 SUC 0342 ',
 'REPOSTERIA XXXX999999XX9 5441 20200923 75326900265000784368680 20200923 suc 0342 CAJA 0093 AUT 01011285 HORA 05:30 CITY ',
 'MARKET 75435290266536760465142 20200923 CAJA 0093 AUT 01011287 20200923 HORA 05:30 SUC 0342 ',
 'FLORES 5992 20200923 75412780265000108700221 20200923 suc 0342 CAJA 0093 AUT 01011289 HORA 05:30 TERMINAL 131210UU7 5811 suc 0342 CAJA 0093 AUT 00881422 HORA 05:30 20200924 75445500266001282364505 20200924 suc 0342 CAJA 0093 AUT 00881422 HORA 05:30 CITY ',
 'MARKET 75435290269536760467046 20200928 CAJA 0093 AUT 00906083 20200928 HORA 05:40 SUC 0342 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451934 HORA 12:19 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451948 HORA 12:32 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0001 AUT 00017534 HORA 11:29 ',
 'UBER.COM 5812 20200928 15276320270002487493963 20200928 suc 0342 CAJA 0093 AUT 02611056 HORA 05:44 1

In [136]:
len(conceptos_pag_2)

11

In [60]:
#retiros
retiros_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('retiros',tabla_pag_2[i].lower()):
        retiros_lista_pag_2.append(tabla_pag_2[i])
        retiros_lista_pag_2.append(tabla_pag_2[i+1])
        retiros_lista_pag_2.append(tabla_pag_2[i+2])
        retiros_lista_pag_2.append(tabla_pag_2[i+3])
        retiros_lista_pag_2.append(tabla_pag_2[i+4])
        retiros_lista_pag_2.append(tabla_pag_2[i+5])
        retiros_lista_pag_2.append(tabla_pag_2[i+6])
        retiros_lista_pag_2.append(tabla_pag_2[i+7])
retiros_lista_pag_2

['RETIROS',
 'DEPÓSITOS',
 '234.00',
 '1 360.00 1,496.20 1 000.00',
 '2,186.73',
 '360.00 8 000.00 4,000.00 15 146.00',
 '1,459.70',
 '493.18']

In [61]:
retiros2_1=[]
for i in retiros_lista_pag_2:
    if re.search(r'\d+', i):
        retiros2_1.append(i)

In [62]:
retiros2_2=[]
for i in retiros2_1:
    retiros2_2.append(re.split(r'\.\d{2}',i))
    
retiros2_2

[['234', ''],
 ['1 360', ' 1,496', ' 1 000', ''],
 ['2,186', ''],
 ['360', ' 8 000', ' 4,000', ' 15 146', ''],
 ['1,459', ''],
 ['493', '']]

In [63]:
retiros2_3=[]
for i in retiros2_2:
    for j in i:
        if re.search(r'\d+', j):
            retiros2_3.append(j+'.00')
        
retiros2_3

['234.00',
 '1 360.00',
 ' 1,496.00',
 ' 1 000.00',
 '2,186.00',
 '360.00',
 ' 8 000.00',
 ' 4,000.00',
 ' 15 146.00',
 '1,459.00',
 '493.00']

In [64]:
len(retiros2_3)

11

In [66]:
#organizo retiros pagina 2
retiros_pag_2=[]
contador=0
for i in range(len(conceptos_pag_2)):
    if conceptos_pag_2[i].startswith('DEPOSITO'):
        retiros_pag_2.append(0)
    
    else:
        retiros_pag_2.append(retiros2_3[contador])
        contador=contador+1
retiros_pag_2

['234.00',
 '1 360.00',
 ' 1,496.00',
 ' 1 000.00',
 '2,186.00',
 '360.00',
 ' 8 000.00',
 ' 4,000.00',
 ' 15 146.00',
 '1,459.00',
 '493.00']

In [152]:
#depositos
depositos_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('depósitos',tabla_pag_2[i].lower()):
        depositos_lista_pag_2.append(tabla_pag_2[i])
depositos_lista_pag_2
#no hay depositos

['DEPÓSITOS']

In [153]:
depositos2=[]
for i in depositos_lista_pag_2:
    if re.search(r'\d+', i):
        depositos2.append(i)

depositos2_1=[]     
for i in depositos2:
    j=i.split(' ')
    for k in j:
        depositos2_1.append(k)

In [154]:
#organizo depositos
depositos_pag_2=[]
contador=0
for i in range(len(conceptos_pag_2)):
    if conceptos_pag_2[i].startswith('DEPOSITO') or conceptos_pag_2[i].startswith('INTERES'):
        depositos_pag_2.append(depositos2_1[contador])
        contador=contador+1
    else:
        depositos_pag_2.append('0')
depositos_pag_2

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

In [75]:
#saldos
saldos_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('saldo',tabla_pag_2[i].lower()):
        saldos_lista_pag_2.append(tabla_pag_2[i])
        saldos_lista_pag_2.append(tabla_pag_2[i+1])
        saldos_lista_pag_2.append(tabla_pag_2[i+2])
        saldos_lista_pag_2.append(tabla_pag_2[i+3])
        
saldos_lista_pag_2

['SALDO 2,715.71 1 355.71 9,859.51 8,859.51',
 '6,672.78 6,312.78 8 312.78 4,312.78 9 166.78',
 '7,707.08',
 '7 213.90']

In [79]:
saldos2_1=[]
for i in range(len(saldos_lista_pag_2)):
    if re.search(r'SALDO \d+', saldos_lista_pag_2[i]):
        saldos2_1.append(saldos_lista_pag_2[i])
        saldos2_1.append(saldos_lista_pag_2[i+1])
        saldos2_1.append(saldos_lista_pag_2[i+2])
        saldos2_1.append(saldos_lista_pag_2[i+3])
        
saldos2_1

['SALDO 2,715.71 1 355.71 9,859.51 8,859.51',
 '6,672.78 6,312.78 8 312.78 4,312.78 9 166.78',
 '7,707.08',
 '7 213.90']

In [80]:
saldos2_2=[]
for i in saldos2_1:
    j=re.split('SALDO|\.\d{2}',i)
    saldos2_2.append(j)
    
saldos2_3=[]
for i in saldos2_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos2_3.append(j)
            
saldos2_3

[' 2,715',
 ' 1 355',
 ' 9,859',
 ' 8,859',
 '6,672',
 ' 6,312',
 ' 8 312',
 ' 4,312',
 ' 9 166',
 '7,707',
 '7 213']

In [82]:
saldos_pag_2=saldos2_3

# pagina 3

In [114]:
tabla_pag_3=pdf[movimientos[2]+1:fin[0]]

In [115]:
#fecha
#fechas
fecha_lista_pag_3=[]
for i in range(len(tabla_pag_3)):
    if re.match('fecha',tabla_pag_3[i].lower()):
        fecha_lista_pag_3.append(tabla_pag_3[i])
    elif re.match(r'\d{2} \w{3}',tabla_pag_3[i]):
        fecha_lista_pag_3.append(tabla_pag_3[i])

        
fecha_lista_pag_3

['FECHA 30 SEP 30 SEP', '02 OCT 02 OCT']

In [116]:
#separo
fechas_lista_2_pag_3=[]

for i in fecha_lista_pag_3:
    fechas_lista_2_pag_3.append(re.findall(r'\d{2} \w{3}',i))

In [117]:
#aplano lista de listas
fechas_pag_3 = []
fechas_pag_3.append(fechas1_pag_3)
for i in fechas_lista_2_pag_3:
    for j in i:
        fechas_pag_3.append(j)

In [118]:
fechas_pag_3

['28 SEP', '30 SEP', '30 SEP', '02 OCT', '02 OCT']

In [126]:
#conceptos
tabla_string_3=" ".join(tabla_pag_3)

In [127]:
conceptos_p3=re.split('COMISI|COB|XXXX|PROSE',tabla_string_3)
conceptos_p3

['FECHA 30 SEP 30 SEP 02 OCT 02 OCT CONCEPTO 20200929 suc 0342 CAJA 0093 AUT 01012606 HORA 05:39 Mexican Peso 20200929 CAJA 0093 AUT 01012606 20200929 HORA 05:39 SUC 0342 ',
 'CCO 20201002 75445500274001277147220 20201002 suc 0342 CAJA 0093 AUT 00684097 HORA 05:25 ',
 '',
 '',
 ' 1311076Y5 7298 suc 0342 CAJA 0093 AUT 00684100 HORA 05:25 20201002 75412910275910272446024 20201002 suc 0342 CAJA 0093 AUT 00684100 HORA 05:25 ',
 'ONOP R 7 CHEQUES GIRADOS CAJA 0000 AUT 01906646 HORA 23:59 SUC 9656 IVA ',
 'ON CHEQUE GIRADO CAJA 0000 AUT 01906646 HORA 23:59 SUC 9656 Centro de Atención Telefónica Ciudad de México 9-999 9999 (1Banamex) Otra ciudad sin costo 99 999 999 9999 RETIROS DEPÓSITOS 351.24 742.90 350.00 105.00 lNecesitas enviar el comprobante de una transferencia? Compártelo después de hacer tu operación desde Citibanamex Móvil · Maneja tu dinero desde la APP. (ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e Página 4 de 11 SALDO 6 862.66 6,119

In [131]:
conceptos_pag_3=[]
for i in range(len(conceptos_p3)):
    if conceptos_p3[i].startswith('FECHA'):
        j=conceptos_p3[i]
        conceptos_pag_3.append(j)
    elif conceptos_p3[i].startswith('CCO'):
        j='PROSE'+conceptos_p3[i]
        conceptos_pag_3.append(j)
    elif conceptos_p3[i].startswith('ONOP '):
        j='COMISI'+conceptos_p3[i]
        conceptos_pag_3.append(j)
    elif conceptos_p3[i].startswith('ON CHEQUE'):
        j='IVA COMISI'+conceptos_p3[i]
        conceptos_pag_3.append(j)
    elif conceptos_p3[i].startswith(' 1311076Y5'):
        j='xxxxxxxxxx'+conceptos_p3[i+1]
        conceptos_pag_3.append(j)
    elif conceptos_p3[i].startswith('COM'):
        j='UBER.'+conceptos_p3[i]
        conceptos_pag_3.append(j)
conceptos_pag_3

['FECHA 30 SEP 30 SEP 02 OCT 02 OCT CONCEPTO 20200929 suc 0342 CAJA 0093 AUT 01012606 HORA 05:39 Mexican Peso 20200929 CAJA 0093 AUT 01012606 20200929 HORA 05:39 SUC 0342 ',
 'PROSECCO 20201002 75445500274001277147220 20201002 suc 0342 CAJA 0093 AUT 00684097 HORA 05:25 ',
 'xxxxxxxxxxONOP R 7 CHEQUES GIRADOS CAJA 0000 AUT 01906646 HORA 23:59 SUC 9656 IVA ',
 'COMISIONOP R 7 CHEQUES GIRADOS CAJA 0000 AUT 01906646 HORA 23:59 SUC 9656 IVA ',
 'IVA COMISION CHEQUE GIRADO CAJA 0000 AUT 01906646 HORA 23:59 SUC 9656 Centro de Atención Telefónica Ciudad de México 9-999 9999 (1Banamex) Otra ciudad sin costo 99 999 999 9999 RETIROS DEPÓSITOS 351.24 742.90 350.00 105.00 lNecesitas enviar el comprobante de una transferencia? Compártelo después de hacer tu operación desde Citibanamex Móvil · Maneja tu dinero desde la APP. (ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e Página 4 de 11 SALDO 6 862.66 6,119.76 5 769.76 5 664.76 0006475 000190.B25INDA0I0.OD.

In [143]:
#retiros
retiros_lista_pag_3=[]
for i in range(len(tabla_pag_3)):
    if re.match('retiros',tabla_pag_3[i].lower()):
        retiros_lista_pag_3.append(tabla_pag_3[i])
        retiros_lista_pag_3.append(tabla_pag_3[i+1])
        retiros_lista_pag_3.append(tabla_pag_3[i+2])
        retiros_lista_pag_3.append(tabla_pag_3[i+3])
        retiros_lista_pag_3.append(tabla_pag_3[i+4])
        retiros_lista_pag_3.append(tabla_pag_3[i+5])
        retiros_lista_pag_3.append(tabla_pag_3[i+6])
        retiros_lista_pag_3.append(tabla_pag_3[i+7])
        retiros_lista_pag_3.append(tabla_pag_3[i+8])
retiros_lista_pag_3

['RETIROS',
 'DEPÓSITOS',
 '351.24 742.90',
 '350.00 105.00',
 'lNecesitas enviar el comprobante de una transferencia?',
 'Compártelo después de hacer tu operación desde Citibanamex Móvil',
 '· Maneja tu dinero desde la APP.',
 '(ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e',
 'Página 4 de 11']

In [146]:
retiros3_1=[]
for i in retiros_lista_pag_3:
    if re.search(r'\d+', i):
        retiros3_1.append(i)

In [147]:
retiros3_2=[]
for i in retiros3_1:
    retiros3_2.append(re.split(r'\.\d{2}',i))
    
retiros3_2

[['351', ' 742', ''],
 ['350', ' 105', ''],
 ['(ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e'],
 ['Página 4 de 11']]

In [148]:
retiros3_3=[]
for i in retiros3_2:
    for j in i:
        if re.search(r'\d+', j):
            retiros3_3.append(j+'.00')
        
retiros3_3

['351.00',
 ' 742.00',
 '350.00',
 ' 105.00',
 '(ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e.00',
 'Página 4 de 11.00']

In [149]:
#organizo retiros pagina 3
retiros_pag_3=[]
contador=0
for i in range(len(conceptos_pag_3)):
    if conceptos_pag_3[i].startswith('DEPOSITO'):
        retiros_pag_3.append(0)
    
    else:
        retiros_pag_3.append(retiros3_3[contador])
        contador=contador+1
retiros_pag_3

['351.00',
 ' 742.00',
 '350.00',
 ' 105.00',
 '(ar¡¡¡ft rISl!os lle ooolooci C01111ll0fl!S y cOfllióa lle ISO d! proikrctDI y lmllOS !11 .e.00']

In [150]:
#depositos
depositos_lista_pag_3=[]
for i in range(len(tabla_pag_3)):
    if re.match('depósitos',tabla_pag_3[i].lower()):
        depositos_lista_pag_3.append(tabla_pag_3[i])
depositos_lista_pag_3
#no hay depositos

['DEPÓSITOS']

In [151]:
depositos3=[]
for i in depositos_lista_pag_3:
    if re.search(r'\d+', i):
        depositos3.append(i)

depositos3_1=[]     
for i in depositos3:
    j=i.split(' ')
    for k in j:
        depositos3_1.append(k)

In [155]:
#organizo depositos
depositos_pag_3=[]
contador=0
for i in range(len(conceptos_pag_3)):
    if conceptos_pag_3[i].startswith('DEPOSITO') or conceptos_pag_3[i].startswith('INTERES'):
        depositos_pag_3.append(depositos3_1[contador])
        contador=contador+1
    else:
        depositos_pag_3.append('0')
depositos_pag_3

['0', '0', '0', '0', '0']

In [156]:
#saldos
saldos_lista_pag_3=[]
for i in range(len(tabla_pag_3)):
    if re.match('saldo',tabla_pag_3[i].lower()):
        saldos_lista_pag_3.append(tabla_pag_3[i])
        saldos_lista_pag_3.append(tabla_pag_3[i+1])
        saldos_lista_pag_3.append(tabla_pag_3[i+2])
        saldos_lista_pag_3.append(tabla_pag_3[i+3])
        
saldos_lista_pag_3

['SALDO 6 862.66', '6,119.76', '5 769.76 5 664.76', '0006475']

In [160]:
saldos3_1=[]
for i in range(len(saldos_lista_pag_3)):
    if re.search(r'SALDO \d+', saldos_lista_pag_3[i]):
        saldos3_1.append(saldos_lista_pag_3[i])
        saldos3_1.append(saldos_lista_pag_3[i+1])
        saldos3_1.append(saldos_lista_pag_3[i+2])
        saldos3_1.append(saldos_lista_pag_3[i+3])


        
saldos3_1

['SALDO 6 862.66', '6,119.76', '5 769.76 5 664.76', '0006475']

In [161]:
saldos3_2=[]
for i in saldos3_1:
    j=re.split('SALDO|\.\d{2}',i)
    saldos3_2.append(j)
    
saldos3_3=[]
for i in saldos3_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos3_3.append(j)
            
saldos3_3

[' 6 862', '6,119', '5 769', ' 5 664', '0006475']

In [162]:
saldos_pag_3= saldos3_3

# juntado

In [164]:
fechas= fechas_pag_1+fechas_pag_2+fechas_pag_3
conceptos=conceptos_pag_1+conceptos_pag_2+conceptos_pag_3
retiros= retiros_pag_1+retiros_pag_2+retiros_pag_3
depositos= depositos_pag_1+depositos_pag_2+depositos_pag_3
saldos= saldos_pag_1+saldos_pag_2+saldos_pag_3


In [165]:
#este data frame se puede exportar como csv
edo_cuenta=pd.DataFrame(list(zip(fechas, conceptos, retiros, depositos, saldos)),
              columns=['fecha','concepto','retiro', 'deposito','saldo'])

In [166]:
edo_cuenta

,fecha,concepto,retiro,deposito,saldo
0,05 SEP,SALDO ANTERIOR,0,0,1 291
1,07 SEP,COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 ...,"2,000.00",0,"9,291"
2,08 SEP,EXCENSION COBRO COMISION MANEJO DE CUENTA PERI...,0,0,"9,291"
3,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,950.00,0,"8,341"
4,10 SEP,COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 ...,1 686.00,0,6 655
5,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,"1,326.00",0,"5,329"
6,18 SEP,GNC PLAZA LILAS MAX0611157H8 9999 999999999999...,"2,029.70.00",0,"3,299"
7,18 SEP,XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJ...,350.00,0,2 949
8,18 SEP,5411 75412910263975987909057 20200921 CAJA 009...,234.00,0,"2,715"
9,21 SEP,REPOSTERIA XXXX999999XX9 5441 20200923 7532690...,1 360.00,0,1 355
